In [45]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
import warnings
warnings.filterwarnings('ignore')

In [46]:
# Reading the Dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [47]:
# We are going to do preprocessing of the dataset
# Droping irrelevent columns
data.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [48]:
# Dividing the dataset into dependent and independent features
X = data.drop('Exited',axis=1)
y = data['Exited']

In [49]:
# Now we are going to do train test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [50]:
data['Gender'].value_counts()

Gender
Male      5457
Female    4543
Name: count, dtype: int64

In [51]:
data['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [52]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot_geo', OneHotEncoder(), ['Geography']),
        ('ordinal_gender', OrdinalEncoder(), ['Gender'])
    ],
    remainder='passthrough'
)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [53]:
# Now we are going to apply standardization to the dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [54]:
X_train

array([[ 1.00150113, -0.57946723, -0.57638802, ...,  0.64920267,
         0.97481699,  1.36766974],
       [-0.99850112,  1.72572313, -0.57638802, ...,  0.64920267,
         0.97481699,  1.6612541 ],
       [-0.99850112, -0.57946723,  1.73494238, ...,  0.64920267,
        -1.02583358, -0.25280688],
       ...,
       [ 1.00150113, -0.57946723, -0.57638802, ..., -1.54035103,
        -1.02583358, -0.1427649 ],
       [ 1.00150113, -0.57946723, -0.57638802, ...,  0.64920267,
        -1.02583358, -0.05082558],
       [-0.99850112,  1.72572313, -0.57638802, ...,  0.64920267,
         0.97481699, -0.81456811]])

In [55]:
# Now we are going to save all the preprocessor and scaler objects using pickle
with open('preprocessor.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

ANN Implementation

In [56]:
X_train.shape

(8000, 12)

In [57]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [58]:
# Building the ANN model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))  #1st hidden layer + input layer
model.add(Dense(32, activation='relu')) #2nd hidden layer
model.add(Dense(1, activation='sigmoid')) #output layer

In [59]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                832       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [60]:
# Compliling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [61]:
# Setup the TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [62]:
# Setup Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [63]:
# Now Start the training of the model
history = model.fit(X_train, y_train,validation_data=(X_test, y_test),epochs=100, batch_size=32, callbacks=[early_stopping_callback, tensorboard_callback])

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 0.4642 - accuracy: 0.7855 - val_loss: 0.3902 - val_accuracy: 0.8365
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3764 - accuracy: 0.8435 - val_loss: 0.3559 - val_accuracy: 0.8555
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3507 - accuracy: 0.8540 - val_loss: 0.3456 - val_accuracy: 0.8605
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3423 - accuracy: 0.8599 - val_loss: 0.3452 - val_accuracy: 0.8590
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3380 - accuracy: 0.8606 - val_loss: 0.3428 - val_accuracy: 0.8605
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3345 - accuracy: 0.8649 - val_loss: 0.3417 - val_accuracy: 0.8635
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3318 - accuracy: 0.8625 - val_loss: 0.3418 - val_accuracy: 0.8610

In [64]:
# Hey Save the model
model.save('churn_model.h5')

In [ ]:
# # Lets Load tensorboard extension and visualize the training process
# import sys
# !{sys.executable} -m tensorboard.main --logdir logs/fit